# Time series to supervised
Before machine learning can be used, time series forecasting problems must be re-framed as supervised learning problems. From a sequence to pairs of input and output sequences as described at https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/

In [1]:
import numpy as np 
import pandas as pd 

import warnings
warnings.filterwarnings('ignore')

In [2]:
def series_to_supervised(data,  col_names, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
     data: Sequence of observations as a list or NumPy array.
     n_in: Number of lag observations as input (X).
     n_out: Number of observations as output (y).
     dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
     Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('%s(t-%d)' % (col_names[j], i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
    if i == 0:
        names += [('%s(t)' % (col_names[j])) for j in range(n_vars)]
    else:
        names += [('%s(t+%d)' % (col_names[j], i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [12]:
CITY = 'LosAngeles'
df = pd.read_pickle('data/%s_selectedfeatures.pkl' % CITY)

In [18]:
df.head()

,demand,dailycoolingdegreedays,dailyheatingdegreedays,hourlydewpointtemperature,hourlydrybulbtemperature,hourlyprecipitation,hourlyrelativehumidity,hourlysealevelpressure,hourlystationpressure,hourlyvisibility,...,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,isbusinessday,isholiday,daylight
datetime,,,,,,,,,,,,,,,,,,,,,
2015-07-01 08:00:00+00:00,3298.0,22.0,0.0,63.0,70.0,0.0,79.0,29.93,29.59,9.0,...,2,3,7,2015,182,1,27,1,0,1
2015-07-01 09:00:00+00:00,3045.0,22.0,0.0,63.0,70.0,0.0,79.0,29.93,29.59,9.0,...,2,3,7,2015,182,1,27,1,0,1
2015-07-01 10:00:00+00:00,2892.0,22.0,0.0,63.0,70.0,0.0,79.0,29.93,29.61,8.7,...,2,3,7,2015,182,1,27,1,0,1
2015-07-01 10:00:00+00:00,2892.0,22.0,0.0,63.0,75.0,0.0,66.0,29.93,29.58,10.0,...,2,3,7,2015,182,1,27,1,0,1
2015-07-01 11:00:00+00:00,2787.0,22.0,0.0,63.0,72.0,0.0,73.0,29.90,29.55,10.0,...,2,3,7,2015,182,1,27,1,0,1


In [20]:
#set the column we want to predict (demand) to the first columns for consistency


values = df.values
# ensure all data is float
values = values.astype('float32')
# frame as supervised learning
reframed = series_to_supervised(values, df.columns, 1, 1)
# drop columns we don't want to predict
reframed.drop(reframed.columns[[15,16,17,18,19,20,21,22,23,24,25,26,27]], axis=1, inplace=True)

cols = list(reframed.columns)
cols.remove('demand(t)')
cols.remove('demand(t-1)')
cols.insert(0,'demand(t)')
reframed = reframed[cols]
reframed.index = df.index[:-1]


In [21]:
reframed.head()

,demand(t),dailycoolingdegreedays(t-1),dailyheatingdegreedays(t-1),hourlydewpointtemperature(t-1),hourlydrybulbtemperature(t-1),hourlyprecipitation(t-1),hourlyrelativehumidity(t-1),hourlysealevelpressure(t-1),hourlystationpressure(t-1),hourlyvisibility(t-1),...,dayofweek(t),quarter(t),month(t),year(t),dayofyear(t),dayofmonth(t),weekofyear(t),isbusinessday(t),isholiday(t),daylight(t)
datetime,,,,,,,,,,,,,,,,,,,,,
2015-07-01 08:00:00+00:00,3045.0,22.0,0.0,63.0,70.0,0.0,79.0,29.93,29.590000,9.0,...,2.0,3.0,7.0,2015.0,182.0,1.0,27.0,1.0,0.0,1.0
2015-07-01 09:00:00+00:00,2892.0,22.0,0.0,63.0,70.0,0.0,79.0,29.93,29.590000,9.0,...,2.0,3.0,7.0,2015.0,182.0,1.0,27.0,1.0,0.0,1.0
2015-07-01 10:00:00+00:00,2892.0,22.0,0.0,63.0,70.0,0.0,79.0,29.93,29.610001,8.7,...,2.0,3.0,7.0,2015.0,182.0,1.0,27.0,1.0,0.0,1.0
2015-07-01 10:00:00+00:00,2787.0,22.0,0.0,63.0,75.0,0.0,66.0,29.93,29.580000,10.0,...,2.0,3.0,7.0,2015.0,182.0,1.0,27.0,1.0,0.0,1.0
2015-07-01 11:00:00+00:00,2790.0,22.0,0.0,63.0,72.0,0.0,73.0,29.90,29.549999,10.0,...,2.0,3.0,7.0,2015.0,182.0,1.0,27.0,1.0,0.0,1.0


In [22]:
# save as pickle file to continue in another notebook
reframed.to_pickle('data/%s_reframed.pkl' % CITY)

In [9]:
reframed.columns

Index(['demand(t-1)', 'dailycoolingdegreedays(t-1)',
       'dailyheatingdegreedays(t-1)', 'hourlydewpointtemperature(t-1)',
       'hourlydrybulbtemperature(t-1)', 'hourlyprecipitation(t-1)',
       'hourlyrelativehumidity(t-1)', 'hourlysealevelpressure(t-1)',
       'hourlystationpressure(t-1)', 'hourlyvisibility(t-1)',
       'hourlywindspeed(t-1)', 'hourlycoolingdegrees(t-1)',
       'hourlyheatingdegrees(t-1)', 'hourlyskyconditions_BKN(t-1)',
       'hourlyskyconditions_CLR(t-1)', 'daylight(t-1)', 'demand(t)',
       'dailycoolingdegreedays(t)', 'dailyheatingdegreedays(t)',
       'hourlydewpointtemperature(t)', 'hourlydrybulbtemperature(t)',
       'hourlyprecipitation(t)', 'hourlyrelativehumidity(t)',
       'hourlysealevelpressure(t)', 'hourlystationpressure(t)',
       'hourlyvisibility(t)', 'hourlywindspeed(t)', 'hourlycoolingdegrees(t)',
       'hourlyheatingdegrees(t)', 'hourlyskyconditions_BKN(t)',
       'hourlyskyconditions_CLR(t)', 'hourlyskyconditions_FEW(t)',
       '